In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import copy
from IPython.display import Image
import qutip as qt
import time
import json

1. Learning rate
2. 

In [23]:
dim = 2      # number of levels for each qubit
GHz = 1e8    #1e9 (0.1GHz)
ns = 1e-9
b = qt.destroy(dim)                     # sigma minus operator

s0m = qt.tensor(b, qt.qeye(dim))           # sigma minus on 0
s0p = qt.tensor(b.dag(), qt.qeye(dim))     # sigma plus on 0
s1m = qt.tensor(qt.qeye(dim), b)           # sigma minus on 1
s1p = qt.tensor(qt.qeye(dim), b.dag())     # sigma plus on 1

omega = [2*np.pi*5.0*GHz, 2*np.pi*5.2*GHz]            # qubit transition frequencies [rad/s]
Delta = [2*np.pi*0*GHz,2*np.pi*0*GHz] #[-.33, -.33]   # anharmonicities [rad/s]
Omega = [2*np.pi*1*GHz, 2*np.pi*1.1*GHz]              # control-qubit drive strength [rad/s]
J = 2*np.pi*0.3*GHz                                   # passive interaction [rad/s]

c_max = 10.0    # upper bound for control amplitudes
c_min = -10.0   # lower bound for control amplitudes
T = 10.0*ns     # target time
L = 10          # number of piecewise constants (at most 20)

H0 = ( omega[0]*s0p*s0m + omega[1]*s1p*s1m ) + \
     (1/2)*( Delta[0]*s0p*s0p*s0m*s0m + Delta[1]*s1p*s1p*s1m*s1m ) + \
     J*(s0p*s1m+s0m*s1p)

Hc0m = Omega[0]*(s0m)
Hc1m = Omega[1]*(s1m)
Hc0p = Omega[0]*(s0p)
Hc1p = Omega[1]*(s1p)

# Piecewise Constant Control on qubit 0
def C0(t, args):
    c = args['c']     # list of control parameters [c11,...,c1L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[0][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[0][L-1]*float(t==t_inv[L])  )

def C1(t, args):
    c = args['c']     # list of control parameters [c21,...,c2L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[1][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[1][L-1]*float(t==t_inv[L])  )

# Piecewise Constant Control on qubit 1
def C2(t, args):
    c = args['c']     # list of control parameters [c31,...,c3L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[2][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[2][L-1]*float(t==t_inv[L])  )

def C3(t, args):
    c = args['c']     # list of control parameters [c41,...,c4L]
    T = args['T']     # target time 
    L = args['L']     # L pieces
    t_inv = list(np.linspace(0,T,L+1))     # endpoints of time interval
    return( np.sum([c[3][i] * ( float(t>=t_inv[i] and t<t_inv[i+1]) ) for i in range(L)])+c[3][L-1]*float(t==t_inv[L])  )

# Marker
def ep0(t, args):
    omega = args['omega']
    return np.exp(1j *omega[0]*t)

def em0(t, args):
    omega = args['omega']
    return np.exp(-1j *omega[0]*t)

def ep1(t, args):
    omega = args['omega']
    return np.exp(1j *omega[1]*t)

def em1(t, args):
    omega = args['omega']
    return np.exp(-1j *omega[1]*t)

# Control Functions C_i_j_k: i=0,1; j=m,p related to sm/sp; k=m,p related to (sm-sp) or (sm+sp)
def C0mm(t, args):
    return 1j*C0(t, args)*ep0(t, args)
def C0pm(t, args):
    return -1j*C0(t, args)*em0(t, args)

def C0mp(t, args):
    return C1(t, args)*ep0(t, args)
def C0pp(t, args):
    return C1(t, args)*em0(t, args)

def C1mm(t, args):
    return 1j*C2(t, args)*ep1(t, args)
def C1pm(t, args):
    return -1j*C2(t, args)*em1(t, args)

def C1mp(t, args):
    return C3(t, args)*ep1(t, args)
def C1pp(t, args):
    return C3(t, args)*em1(t, args)

# Total Hamiltonain
H = [H0, [Hc0m,C0mm], [Hc0p,C0pm], [Hc0m,C0mp], [Hc0p,C0pp], [Hc1m,C1mm], [Hc1p,C1pm], [Hc1m,C1mp], [Hc1p,C1pp]]   


GA = qt.sigmax() # Target Unitary on qubit 0
GB = qt.sigmax() # Target Unitary on qubit 1

In [10]:
%whos

Variable          Type             Data/Info
--------------------------------------------
C0                function         <function C0 at 0x0000019AFEFDF790>
C0mm              function         <function C0mm at 0x0000019AFEFDF940>
C0mp              function         <function C0mp at 0x0000019AFEFDFB80>
C0pm              function         <function C0pm at 0x0000019AFEFDF8B0>
C0pp              function         <function C0pp at 0x0000019AFEFDF4C0>
C1                function         <function C1 at 0x0000019AFEFDFF70>
C1mm              function         <function C1mm at 0x0000019AFEFDF550>
C1mp              function         <function C1mp at 0x0000019AFEFDFDC0>
C1pm              function         <function C1pm at 0x0000019AFEFDFD30>
C1pp              function         <function C1pp at 0x0000019AFEFDF040>
C2                function         <function C2 at 0x0000019AFEFDF670>
C3                function         <function C3 at 0x0000019AFEFDF310>
Delta             list             n=2
GA 

In [12]:
print(Delta)
print(Omega)
print(omega)

[0.0, 0.0]
[628318530.7179586, 691150383.7897546]
[3141592653.589793, 3267256359.7333846]


In [30]:
def OBJ(c,T,L,omega,H,GA,GB):
    
    args = {
        'c': c,
        'T': T,
        'L': L,
        'omega': omega
    }
    
    U = qt.propagator(H, T, args=args, options=qt.Options(nsteps=1700))
    
    # SVD for partial traces
    sigma = np.sqrt((((qt.tensor(GA, qt.qeye(dim)).dag()*U).ptrace(1))*((qt.tensor(GA, qt.qeye(dim)).dag()*U).ptrace(1)).dag()).eigenstates()[0])
    lambd = np.sqrt((((qt.tensor(qt.qeye(dim), GB).dag()*U).ptrace(0))*((qt.tensor(qt.qeye(dim), GB).dag()*U).ptrace(0)).dag()).eigenstates()[0])
    
    # sum of fidelity
    return sum(sigma)/(2*dim**2)+sum(lambd)/(2*dim**2)

In [14]:
%whos

Variable          Type             Data/Info
--------------------------------------------
C0                function         <function C0 at 0x0000019AFEFDF790>
C0mm              function         <function C0mm at 0x0000019AFEFDF940>
C0mp              function         <function C0mp at 0x0000019AFEFDFB80>
C0pm              function         <function C0pm at 0x0000019AFEFDF8B0>
C0pp              function         <function C0pp at 0x0000019AFEFDF4C0>
C1                function         <function C1 at 0x0000019AFEFDFF70>
C1mm              function         <function C1mm at 0x0000019AFEFDF550>
C1mp              function         <function C1mp at 0x0000019AFEFDFDC0>
C1pm              function         <function C1pm at 0x0000019AFEFDFD30>
C1pp              function         <function C1pp at 0x0000019AFEFDF040>
C2                function         <function C2 at 0x0000019AFEFDF670>
C3                function         <function C3 at 0x0000019AFEFDF310>
Delta             list             n=2
GA 

In [94]:
def GRAPE(T,L,omega,H,GA,GB,c=None):
    eps = 0.5               # ascending rate
    dx = 0.001              # gradient increment
    threshold = 0.000001    # if cost changes less than threshold, then halt
    itera_max = 25          # max iteration number
    count = 0               # counting iterations
    diff_cost = np.inf      # initial cost difference
    d = T/L                 # time interval length
    record_cost = []
    factor = 0.5
    
    # Step1-1, Guess initial
    if c is None:
        c = [[float(np.random.uniform(c_min,c_max)) for _ in range(L)],    # for C1
             [float(np.random.uniform(c_min,c_max)) for _ in range(L)],    # for C2
             [float(np.random.uniform(c_min,c_max)) for _ in range(L)],    # for C3
             [float(np.random.uniform(c_min,c_max)) for _ in range(L)]]    # for C4
        
    # Step1-2, Compute initial cost
    cost = OBJ(c,T,L,omega,H,GA,GB)
    record_cost.append(cost)
    # Start while loop
    
    while (count<=itera_max) and (diff_cost>threshold):
        count += 1
        # Step 2, compute derivative numerically by Df(x0) = (f(x0+dx)-f(x0)) / dx
        D_J_c = [ [ 0 for l in range(L)] for m in range(4)]
        for m in range(4):
            for l in range(L):
                c_tmp1 = copy.deepcopy(c)
                c_tmp2 = copy.deepcopy(c)
                c_tmp1[m][l] += dx/2
                c_tmp2[m][l] -= dx/2
                D_J_c[m][l] += (OBJ(c_tmp1,T,L,omega,H,GA,GB) - OBJ(c_tmp2,T,L,omega,H,GA,GB)) / dx
        # Step 3, normalization for gradient
        norm2_D = 0
        for m in range(4):
            for l in range(L):
                norm2_D += D_J_c[m][l]**2
        norm_D = np.sqrt(norm2_D)
        D_J_c = [[eps*D_J_c[m][l]/norm_D for l in range(L)] for m in range(4)]
        # Step4, gradient ascend
        c_old = np.array(c)
        c = np.array(c) + np.array(D_J_c)
        # Step5, Compute cost
        new_cost = OBJ(c,T,L,omega,H,GA,GB)
        
        if new_cost > record_cost[-1]:
            diff_cost = abs(new_cost - record_cost[-1])
            record_cost.append(new_cost)
        else:
            c = c_old
            dx *= factor
            eps *= factor
            
    return record_cost, c, count

In [36]:
%whos

Variable          Type             Data/Info
--------------------------------------------
C0                function         <function C0 at 0x0000019AFFE1FAF0>
C0mm              function         <function C0mm at 0x0000019AFFE1F1F0>
C0mp              function         <function C0mp at 0x0000019AFFE1F550>
C0pm              function         <function C0pm at 0x0000019AFFE1F3A0>
C0pp              function         <function C0pp at 0x0000019AFFE1F8B0>
C1                function         <function C1 at 0x0000019AFFE1FC10>
C1mm              function         <function C1mm at 0x0000019AFFE1F940>
C1mp              function         <function C1mp at 0x0000019AFFE1FDC0>
C1pm              function         <function C1pm at 0x0000019AFFE1F040>
C1pp              function         <function C1pp at 0x0000019AFFE1F280>
C2                function         <function C2 at 0x0000019AFFE1F4C0>
C3                function         <function C3 at 0x0000019AFFE1F5E0>
Delta             list             n=2
GA 

In [33]:
n_trails = 1
record_c = [{} for i in range(n_trails)]
record_f = [{} for i in range(n_trails)] # array of dict

for i in range(n_trails): # apply GRAPE with random initial multiple times
    print('i=',i) # check progress
    fidelity,c = GRAPE(T,L,omega,H,GA,GB)
    record_f[i].update({i: fidelity})
    record_c[i].update({i: c.tolist()})
        
    with open("f{}.json".format(i),'w') as outfile:
         json.dump(record_f, outfile)
    with open("c{}.json".format(i),'w') as outfile:
         json.dump(record_c, outfile)
            
    print(record_f[0][0][-1])

i= 0
0.5190024481840829
0.893069909822491
0.8168802985846182
0.8950558273365075
0.8602937055578981
0.8962277035119184
0.8937156674281109
0.8976642303446455
0.9189420932422148
0.9203497272470322
0.9359715676603073
0.9360011852807295
0.9360011852807295


In [37]:
fidelity,c = GRAPE(T,L,omega,H,GA,GB,c)

0.9360011852807295
0.703098987605711
0.9370468088964712
0.9354783521233653
0.9383950643584456
0.9686132107810543
0.9702354156183892
0.9710378498712875
0.9716260662262394
0.9719015973283958
0.9720408804032675
0.9721046989257885


In [38]:
fidelity,c = GRAPE(T,L,omega,H,GA,GB,c)

0.9721046989257885
0.9671590813570836
0.7287072992785137
0.958719064795936
0.9557802003126967
0.9590649869439969
0.9877598373446497
0.9864989849408093
0.9876305289876574
0.9892450214247198
0.9894966852063296
0.9896652048061663


In [39]:
n_trails = 1
record_c = [{} for i in range(n_trails)]
record_f = [{} for i in range(n_trails)] # array of dict

for i in range(n_trails): # apply GRAPE with random initial multiple times
    print('i=',i) # check progress
    fidelity,c = GRAPE(T,L,omega,H,GA,GB)
    record_f[i].update({i: fidelity})
    record_c[i].update({i: c.tolist()})
        
    with open("f{}.json".format(i),'w') as outfile:
         json.dump(record_f, outfile)
    with open("c{}.json".format(i),'w') as outfile:
         json.dump(record_c, outfile)
            
    print(record_f[0][0][-1])

i= 0
0.48079747183416977
0.8736454915983105
0.9288437058323058
0.9473598170946306
0.9543788909340225
0.9573555934333361
0.9587071897470784
0.9593415404672919
0.9596524068230425
0.9598040403442275
0.959880095390975
0.9599155172866964
0.9599155172866964


In [45]:
fidelity,c = GRAPE(T,L,omega,H,GA,GB,c)

0.9599155172866964
0.9583107184284692
0.9583107184284692


In [50]:
fidelity,c = GRAPE(T,L,omega,H,GA,GB,c)

0.9599155172866964
0
1
0.9694433656982304
2
0.973072450528258
3
4
5
0.9782891441018313
6
0.9788103081011119
7
0.9802195188686906
8
0.9805252368797276
9
0.981517349023537
10
0.98176462970392


In [ ]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)

In [87]:
n_trails = 25
record_c = [{} for i in range(n_trails)]
record_f = [{} for i in range(n_trails)] # array of dict

for i in range(n_trails): # apply GRAPE with random initial multiple times
    print('i=',i) # check progress
    fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB)
    record_f[i].update({i: fidelity})
    record_c[i].update({i: c.tolist()})
        
    with open("f{}.json".format(i),'w') as outfile:
         json.dump(record_f, outfile)
    with open("c{}.json".format(i),'w') as outfile:
         json.dump(record_c, outfile)
            
    print(record_f[i][i][-1])

i= 0
0.8898849027943891
i= 1
0.8898849027943891
i= 2
0.8898849027943891
i= 3
0.8898849027943891
i= 4
0.8898849027943891
i= 5
0.8898849027943891
i= 6
0.8898849027943891
i= 7
0.8898849027943891
i= 8
0.8898849027943891
i= 9
0.8898849027943891
i= 10
0.8898849027943891
i= 11
0.8898849027943891
i= 12
0.8898849027943891
i= 13
0.8898849027943891
i= 14
0.8898849027943891
i= 15
0.8898849027943891
i= 16
0.8898849027943891
i= 17
0.8898849027943891
i= 18
0.8898849027943891
i= 19
0.8898849027943891
i= 20
0.8898849027943891
i= 21
0.8898849027943891
i= 22
0.8898849027943891
i= 23
0.8898849027943891
i= 24
0.8898849027943891


In [88]:
for i in range(25):
    print(record_f[i][i][-1])
    if record_f[i][i][-1]<0.8:
        print('')
        print(i)
        print(record_f[i][i])
        print('')

0.8898849027943891
0.915862034643798
0.9494842202369964
0.9500674903262853
0.9119126864935637
0.8893088848804723
0.8903050144948823
0.9301732289336877
0.9428815703575393
0.9374661069280372
0.9397281824608049
0.9279326105560846
0.9080804670619705
0.9233675594710959
0.9537320741381932
0.9288470961560316
0.9177937671981806
0.9271889194881543
0.948908094928411
0.8915066543034957
0.9437754550836559
0.93894304996173
0.8782519345586761
0.9546961485844029
0.9611758167430436


In [93]:
n_trails = 25
record_c = [{} for i in range(n_trails)]
record_f = [{} for i in range(n_trails)] # array of dict

for i in range(n_trails): # apply GRAPE with random initial multiple times
    print('i=',i) # check progress
    fidelity, c, counts = GRAPE(T,L,omega,H,GA,GB)
    record_f[i].update({i: fidelity})
    record_c[i].update({i: c.tolist()})
        
    with open("f{}.json".format(i),'w') as outfile:
         json.dump(record_f, outfile)
    with open("c{}.json".format(i),'w') as outfile:
         json.dump(record_c, outfile)
            
    print(record_f[i][i][-1])

i= 0
0.9860965137762707
i= 1
0.6209256041233735
i= 2
0.9859277221614906
i= 3
0.6364995578981831
i= 4
0.9820626586606792
i= 5
0.9687414378749342
i= 6
0.9682730138107294
i= 7
0.958885704650666
i= 8
0.9728321115237715
i= 9
0.9668644574082936
i= 10
0.6390892546331411
i= 11
0.4322210070165903
i= 12
0.6612376966219354
i= 13
0.9663435702991445
i= 14
0.9404458200532158
i= 15
0.636434309247029
i= 16
0.6223194813045843
i= 17
0.9720072329414458
i= 18
0.6056593005271016
i= 19
0.619508756213704
i= 20
0.9721196674617589
i= 21
0.9830088335770453
i= 22
0.650708961133629
i= 23
0.991631995583075
i= 24
0.9696290546856485


In [ ]:
n_trails = 25
record_c = [{} for i in range(n_trails)]
record_f = [{} for i in range(n_trails)] # array of dict

for i in range(n_trails): # apply GRAPE with random initial multiple times
    print('i=',i) # check progress
    fidelity, c, counts = GRAPE(T,L,omega,H,GA,GB)
    record_f[i].update({i: fidelity})
    record_c[i].update({i: c.tolist()})
        
    with open("f{}.json".format(i),'w') as outfile:
         json.dump(record_f, outfile)
    with open("c{}.json".format(i),'w') as outfile:
         json.dump(record_c, outfile)
            
    print(record_f[i][i][-1])

i= 0
0.6184355868623852
i= 1
0.9685176342281363
i= 2
0.6432638482641587
i= 3
0.9790440492116214
i= 4
0.6086331828859917
i= 5
0.9708879159360603
i= 6
0.4976648096577976
i= 7
0.9481553464205745
i= 8
0.9797213234894442
i= 9
0.6311939638554174
i= 10
0.9746185823067152
i= 11
0.9796234246579189
i= 12
0.9888452589950483
i= 13
0.6598355711044035
i= 14
0.969806038160521
i= 15
0.9808277232353875
i= 16
0.632640417804358
i= 17
0.9820214111538441
i= 18
0.9521517962178083
i= 19
0.9560602472599384
i= 20
0.9764074243080365
i= 21


In [84]:
local_c0 = record_c[12][12]
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,local_c0)
fidelity

[0.7757630805454179, 0.9164428828337072, 0.9431835649356624]

In [85]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)
fidelity

[0.9431835649356624, 0.9613352100474241]

In [86]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)
fidelity

[0.9613352100474241, 0.9678305476627796]

In [78]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)
fidelity

[0.664802274705858, 0.6665805177238705]

In [79]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)
fidelity

[0.6665805177238705, 0.6734431321129253]

In [80]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)
fidelity

[0.6734431321129253]

In [83]:
fidelity,c, counts = GRAPE(T,L,omega,H,GA,GB,c)
fidelity

[0.6734431321129253]

In [15]:
n_trails = 2
record_c = [{} for i in range(n_trails)]
record_f = [{} for i in range(n_trails)] # array of dict

for i in range(n_trails): # apply GRAPE with random initial multiple times
    print('i=',i) # check progress
    fidelity,c = GRAPE(T,L,omega,H,GA,GB)
    record_f[i].update({i: fidelity})
    record_c[i].update({i: c.tolist()})
    
    
    with open("f1{}.json".format(i),'w') as outfile:
         json.dump(record_f, outfile)
    with open("c1{}.json".format(i),'w') as outfile:
         json.dump(record_c, outfile)
            
    print(record_f[i][i][-1])

i= 0
0.8982411397693636
i= 1
0.8982411397693636


In [19]:
print(record_f[1][1][-1])
print(record_f[0][0][-1])

0.6211406455239948
0.8982411397693636


In [8]:
with open('f11.json') as json_file:
    print((json.load(json_file))[1])
with open('c11.json') as json_file:
    local_c0 = (json.load(json_file))[1]['1']
    print(local_c0)

{'1': [0.3595293431930304, 0.40632504718213236, 0.4447530113914113, 0.4763650503197419, 0.502084843106714, 0.5231001603356871, 0.5403351667876334, 0.5543996449274524, 0.5659732061462335, 0.5752767274971682, 0.5831394472983944, 0.5896073190804272, 0.5948687276024519, 0.5991128006766401, 0.6027314868279712, 0.6056930174215357, 0.6080763520748425, 0.6102434621942561, 0.6120379139359727, 0.6134635679346114, 0.6146863295704392, 0.6156485233207074, 0.6166119767596396, 0.6172155661583489, 0.617959246854076, 0.6186386211591408, 0.6191794515036135, 0.6194674056895145, 0.6197585305453764, 0.6199573119263949, 0.6201799811815873, 0.6203362255422202, 0.6205277621229689, 0.6205915649964578, 0.6206312046978876, 0.6207015106342078, 0.620749795121307, 0.6208493902447438, 0.6209668649820024, 0.6209994353007874, 0.621010931864555, 0.6210488799388538, 0.621072743673303, 0.6210757834956055, 0.6210915379903552, 0.62109949207281, 0.6211088795442717, 0.6210787168495131, 0.6210951097176176, 0.6211118751572624,

In [16]:
local_c0 = c # then run the above block again, and the fidelity will keep changing

In [17]:
fidelity,c = GRAPE(T,L,omega,H,GA,GB,local_c0)
fidelity

[0.9659896535888226,
 0.9816722349196898,
 0.5508711917707093,
 0.9624714899444038,
 0.6774300139403051,
 0.9606979836610423,
 0.7462667450496959,
 0.9610227375298154,
 0.8025763469670382,
 0.9613389355360886,
 0.8482454132659102,
 0.9616300111586473,
 0.8847925851318472,
 0.9618745226846048,
 0.913596799093056,
 0.9622306681264793,
 0.935924456674663,
 0.9625020722156127,
 0.9529879590437909,
 0.9627907662410784,
 0.9658297517145262,
 0.9727461437972915,
 0.9744821084305668,
 0.9794017237444693,
 0.9803045861796268,
 0.983890400807448,
 0.9843209253306549,
 0.9868951926112044,
 0.9870810944874011,
 0.9889641952596124,
 0.9890179610566718,
 0.9904001490672516,
 0.990398666433753,
 0.9907193816692839,
 0.9914642172439471,
 0.9916930915867244,
 0.9922559762541259,
 0.9924310608244761,
 0.9928577534725769,
 0.9930084112291957,
 0.9933267313263461,
 0.9934747735082304,
 0.9937223275407119,
 0.9938712966465043,
 0.9940658045587909,
 0.9942157263510076,
 0.9943719785946518,
 0.99451104364977